In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import tqdm
import firedrake
from firedrake import Constant, as_vector, interpolate, project
import plumes, plumes.coefficients

# Plumes

This notebook will demonstrate the plume solver on a synthetic geometry.
The prognostic variables that describe the plume are:

* thickness $D$
* velocity $u$
* temperature $T$
* salinity $S$

The variables that characterize the external environment are:

* ice shelf draft $z_b$
* ambient ocean temperature $T_a$
* ambient ocean salinity $S_a$

The model also prescribes several auxiliary quantities as a function of both the plume state and ambient ocean state:

* relative density contrast $\delta\ln\rho$ between plume and ocean: a linear function of plume and ocean salinity and temperature
* freezing temperature $T_f$: linear function of salinity and ice shelf draft
* melt rate $\dot m$ from ice shelf base: a function of freezing temperature, temperature, velocity
* entrainment rate $\dot e$ of background ocean water: a function of plume velocity and ice shelf draft

We will spin the plume up to a steady state in several phases:

1. hold the $T$ and $S$ fixed and spin up $D$ and $u$
2. hold $D$ and $u$ fixed and spin up $T$ and $S$
3. spin up all variables

### Geometry and input data

The domain will be a 20km x 20km ice shelf cavity.
We'll use 64 quads on each side and we'll use degree-1 discontinuous basis functions.

In [ ]:
Lx, Ly = 20e3, 20e3
nx, ny = 64, 64
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, quadrilateral=True)
Z = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='DQ', degree=0)
V = firedrake.VectorFunctionSpace(mesh, family='DQ', degree=0)

We'll assume that the background ocean has a temperature of $T = -1.91^\circ$C and a salinity of $S = 34.65$ psu.
These values come from Lazeroms 2018.
(The HSSW subscript stands for high-salinity shelf water.)

In [ ]:
T_hssw = Constant(-1.91)  # degrees C
S_hssw = Constant(34.65)  # PSU

Likewise these values are typical of melted ice shelf water (ISW), which is very cold and has virtually zero salt content.

In [ ]:
T_isw = Constant(-3.5)
S_isw = Constant(0.)

Now to make some synthetic initial data.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

The ice shelf will be linearly sloping from a depth of 600m to a depth of 400m over the entire domain.

In [ ]:
z_in = Constant(-600.)
z_out = Constant(-300.)
X = x[0] / Constant(Lx)
z_b = interpolate((1 - X) * z_in + X * z_out, Z)

We have to fix the inflow thickness and velocity of the plume.
**The inflow values are a form of boundary forcing and they never change.**

In [ ]:
D_in = Constant(.1)
u_in = Constant(.01)

Next we have to make some kind of initial guess for the plume thickness and velocity everywhere.
There's no good way to do this.

In [ ]:
δD = Constant(5.)
δu = Constant(0.1)

In [ ]:
D0 = interpolate(D_in + δD * X, Q)
u0 = interpolate(as_vector((u_in + δu * X, 0.)), V)

Finally, we'll guess that the initial salinity and temperature of the plume is about 90\% towards that of the ambient ocean and 10\% towards that of just-melted ice shelf water.
I've chosen these values so that the initial temperature exceeds the freezing temperature, which will make the plume grow in thickness.

In [ ]:
a = Constant(0.1)
T = Constant((1 - a) * T_hssw + a * T_isw)
S = Constant((1 - a) * S_hssw + a * S_isw)

print(float(T), float(S))

The object called `model` that we create below is responsible for calculating all of the things that we listed at the start of this notebook -- the melt rate, the entrainment rate, the density contrast, as well as the fluxes and sources for all of the dynamical variables.

In [ ]:
model = plumes.PlumeModel()

We'll use this now to calculate a few of these fields to check that the values are sane.

In [ ]:
freezing_temp_in = float(model.freezing_temperature(salinity=S, ice_shelf_base=z_in))
freezing_temp_out = float(model.freezing_temperature(salinity=S, ice_shelf_base=z_out))
print('Freezing temperature at:\n  inflow: {}\n  outflow: {}'
      .format(freezing_temp_in, freezing_temp_out))

The melt rate is positive but just barely -- the units below are in meters / year.

In [ ]:
melt = project(model.melt(velocity=u0, temperature=T, salinity=S, ice_shelf_base=z_b), Z)
print(melt.at((Lx / 2, Ly / 2)) * 365.25 * 60 * 60)

Finally let's make sure that the plume is buoyant by checking that the relative density contrast is positive.

In [ ]:
δlnρ = model.density_contrast(
    temperature=T,
    salinity=S,
    temperature_ambient=T_hssw,
    salinity_ambient=S_hssw
)
print(float(δlnρ))

Finally we'll interpolate the salinity and temperature values to actual fields defined in the finite element space.

In [ ]:
S0 = interpolate(S, Q)
T0 = interpolate(T, Q)

### Initial spin-up

Once we've created all the state variables and sources, we'll pack them into some dictionaries to pass to the solvers.
These are:

* `fields`: the dynamical variables of the system
* `inflow`: boundary conditions; what's coming in from the grounding line
* `inputs`: external state that interacts with the plume, including the shape of the ice shelf and the background ocean properties

We could pass all these as keywords but this is more concise.

In [ ]:
fields = {
    'thickness': D0.copy(deepcopy=True),
    'velocity': u0.copy(deepcopy=True),
    'temperature': T0.copy(deepcopy=True),
    'salinity': S0.copy(deepcopy=True)
}

inflow = {
    'thickness_inflow': D0,
    'velocity_inflow': u0,
    'temperature_inflow': T0,
    'salinity_inflow': S0
}

inputs = {
    'ice_shelf_base': z_b,
    'salinity_ambient': S_hssw,
    'temperature_ambient': T_hssw
}

Now we'll create the solver object.
The model object only tells us what problem it is we're dealing with; the solver actually computes things.
We'll pass some flags to the solver to tell it to only update the plume thickness and momentum.

In [ ]:
from plumes import Component
components = Component.Mass | Component.Momentum
solver = plumes.PlumeSolver(
    model, components,
    **fields, **inflow, **inputs
)

Now we choose a timestep that satisfies the Courant-Friedrichs-Lewy condition.
This is challenging to do a priori -- the velocity increases during the simulation, so an initially good choice can become bad later on.
You could tune this automatically but for now I've just done some trial and error.
Once we've found a steady state we can diagnose it properly.

In [ ]:
δx = Lx / (nx + 1)
timestep = δx / float(u_in + δu) / 128
print(f'Timestep: {timestep}')

In [ ]:
num_days = 1.
final_time = num_days * 24 * 60 * 60
num_steps = int(final_time / timestep)
print(f'Number of steps: {num_steps}')
dt = final_time / num_steps

In [ ]:
pbar = tqdm.trange(num_steps)
for step in pbar:
    solver.step(dt)
    D = solver.fields['thickness']
    Dmin = D.dat.data_ro.min()
    Dmax = D.dat.data_ro.max()
    if step % 50 == 0:
        pbar.set_description('thickness min/max: ({:4.2f}, {:4.2f})'
                             .format(Dmin, Dmax))

The max velocity at the end of the simulation is on the high side for global ocean circulation but not completely ridiculous.

In [ ]:
umax = solver.fields['velocity'].dat.data_ro[:, 0].max()
print(f'max velocity: {umax}')

The new timestep that works well with the CFL condition would be:

In [ ]:
timestep = δx / umax / 8
num_steps = int(final_time / timestep)
dt = final_time / num_steps
print(f'Timestep: {dt}')

### Secondary spin-up

Now we'll run the salinity and temperature to steady state, holding the thickness and velocity fixed.
To do this, we'll change the `components` flag in the solver object.

In [ ]:
print(f'Before: {solver.components}')
solver.components = Component.Heat | Component.Salt
print(f'After: {solver.components}')

And now for the show:

In [ ]:
pbar = tqdm.trange(num_steps)
for step in pbar:
    solver.step(dt)
    T = solver.fields['temperature']
    Tmin = T.dat.data_ro.min()
    Tmax = T.dat.data_ro.max()
    if step % 50 == 0:
        pbar.set_description('temperature min/max: ({:4.2f}, {:4.2f})'
                             .format(Tmin, Tmax))

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
tris = firedrake.tripcolor(solver.fields['temperature'], num_sample_points=24, axes=axes)
fig.colorbar(tris)

### Final spin-up

Now we can add all components and pray.

In [ ]:
solver.components = Component.All

num_days = 2.
final_time = num_days * 24 * 60 * 60
timestep = δx / umax / 8
num_steps = int(final_time / timestep)
dt = final_time / num_steps

In [ ]:
import numpy as np

us = np.zeros(num_steps)
Ds = np.zeros((num_steps, 2))
Ts = np.zeros_like(Ds)
Ss = np.zeros_like(Ds)

pbar = tqdm.trange(num_steps)
for step in pbar:
    solver.step(dt)
    u = solver.fields['velocity']
    D = solver.fields['thickness']
    T = solver.fields['temperature']
    S = solver.fields['salinity']
    
    us[step] = np.sqrt(np.sum(u.dat.data_ro**2, axis=1)).max()
    Ds[step, :] = D.dat.data_ro.min(), D.dat.data_ro.max()
    Ts[step, :] = T.dat.data_ro.min(), T.dat.data_ro.max()
    Ss[step, :] = S.dat.data_ro.min(), S.dat.data_ro.max()

    if step % 50 == 0:
        pbar.set_description('velocity max: {:4.2f}'.format(us[step]))

In [ ]:
fig, axes = plt.subplots()
axes.plot(us)

In [ ]:
fig, axes = plt.subplots()
axes.plot(Ds[:, 0])
axes.plot(Ds[:, 1])

In [ ]:
fig, axes = plt.subplots()
axes.plot(Ss[:, 0])
axes.plot(Ss[:, 1])